# Bockudoku and Deep Q: State Space Compression Strategies
## By Connor Haugh for COSC 247: Machine Learning
## Fall 2020

Part 0: What's in this file?: This notebook contains results and analysis of my project-- the actual models and enviroments are in seperate python files in this folder. If you are interested in how these things are implmented, look to those files.

# <span style="color:blue"> Part 1: The Enviroment</span>


### What is BlockuDoku? 

BlockuDoku is a unique combination of sudoku, tetris, and Blockus, which has fascinated me because I cannot improve my performance despite playing it almost weekly. The game begins with a blank 9x9 grid, and you are given three tiles of different sizes (1-5 grid squares) at once. You must fit these three tiles one at a time onto the grid. If no tiles can be placed, you lose the game. Filling a row or a traditional sudoku “box” causes all elements inside that row or box to disappear, allowing more tiles to be placed. “Score” is determined by the following parameters: 1 point for each tile placed, and 2 points for removing a tile. Exploring how to most effectively solve this game will not only produce a solution to the game, but also explore the connections between game-playing and reinforcement learning.

BlockuDoku poses a challenge in one respect: the state space is very large. If we think of the grid or "board" as a nine by nine binary matrix, where a 1 repersents a placed tile, and 0 an empty space, the possible states of just the board are 2^81. Therefore, as states are unlikely to repeat themselves, search-based appraoches may fall short.

As a general strategy, I hope to employ various forms of reinforcement learning to learn about their effectiveness. From an AI perspective, we can think about the kinds of learning models which might learn effectively. Search-based approaches will quickly become too complex, as the state space for a particular turn is large (on the first turn, there are about 50 next states). Accounting for uncertainty, namely what pieces will be generated next, will allow for stateless generalizable learners to perform better. The Reward function will merely be the score of the game, features will include the number of tiles on the board, the number of grid locations which are “isolated” or can only be filled by a 1-tile piece and other features of the board.

I intend to, in order to demonstrate not only my existing knowledge of q-learning, but expand this knowledge into the realm of machine learning, I will use both a “basic” q-learning approach, as well as a “deep” approach which maintains a NN instead of just a weight vector, which will hopefully allow for not only improved performance, but also my one increased understanding of how NNs work. I will also evaluate based on the performance of the base q-learning model and “deep-q” whether or not the performance of the model is improved by this model. I hope to document this using a nice visualiation with matpolitlib which will show the progress of games in animation format.



# <span style="color:blue"> Part 2: The Approach</span>

How should we apply existing techniques to produce an agent which solves this problem? The first thing I turned to were solutions to similair problems. I read Stevens and Pradhan, who solved tetris with a deep reenforcement learning approach. Blockuduku is probably most similair to tetris in its game rules, so I thought to work my way there.

We have to think about the obvious challenges of solving this probelm. The first is the very large state space-- there are two approaches we can use to solve this using reenforcement learning, which is one of the only Machine Learning model which does not require a prebuilt dataset.

We can:
    - solve the vast state space by creating a set of features of a given game state. A feature might be the number of open white squares. Instead of learning on states, we can learn on the exrapolated features from the states to do "generalized" q-learning.
    - solve the vast state space problem by using a neural net, which inside "the black box" of hidden layers, might be able to parametrize the problem much more effectively than we can.

What are the potential drawbacks of these solutions?

    -for neural nets, we must account for "overfitting" in which random piece generation provides a bunch of changing options, and we don't want to optimize to local exterema. Instead, we should learn on a generalized version of a large time period. This is where Lin's solution of "experince replay" is helpful. Instead of learning on every experince, instead only train the neural net on a random selection of the last "batch" of a hundered or so experinces. This way, over training, the net will learn more slowly, but achieve better results.
    -for regular "generalized" Q learning, one of the biggest challenges will be determineing features which do not directly correllate to JUST the reward at a given state.






## <span style="color:blue"> Explaining things: Q and Deep Q</span>

Q learning is an approach to learning not on a dataset, but an enviroment. A "Q" value is asrcibed to every state,action pair in the enviroment, and can be understood as a belief about that pair. As the agent experinces the world, it encounters these state,action pairs, and updates its beliefs based on the given experince throught the Bellman equation, which updates the Q value based on the "reward" given for performing that action at that state-- running into the endzone for a touchdown might give a good reward, and therefore bumps up the q vlaue. Dropping the ball on the one yard line, however, might have a negative reward, and reduce the "value" of performing that action at that state. Upon repeated exploration of the state, an agent will be able to be given a state, and idenitfy the corresponding action which seems most reasonable.

"Deep" Q learning combines the bellman update equation with a gradient desncent equation. That way, input states can be used to tune a neuarl net, which then makes predictions. In order to speed things up, I took something I had seen before from chess AI. It would be very difficult to create a neuarl net which produced a numerical output which mapped to a particular action to take. Therefore, our model merely takes in states and evaluates them on a scale of zero to one hundred based on experince. If we just input states with the action already applied (in this case the blocks placed on the grid) then we jsut rate the states, and don't ahve to worry about the legality of our outputs anymore. To select the value, all we have to do therefore is tkae the maximum rated result. I think this is a really helpful and novel idea, which I might want to explore further.

## NOTE: A performance upgrade:
    
    In order to most effectively solve this problem in an expediant amount of time, I reduced the size of the grid from 9X9 to 6X6. This might divert from the original goals of the project, but makes it feasible within the time period (a larger state space dramitaclly increased runtime).



## Visualizing the Models





# PART 2: THE EXPERIMENT

The respective random models were trained on a differing number of games. THe random agent only played 1000 games, becasue that seemed like neough to model average performance, while the Q learning agent performed 5,000 games. 

## HyperParameters: 
       Q Learning (Deep and Shallow) has several parameters
           * Exploration Rate : Determines how often a random action is taken instead of the highest Q value to promote exploration of the state space. A value is 1/10.
           * Learning rate (alpha) : determines the extent to which a given example will bump up/down the q value based on the reward. Learning rate had negligable effects on performance, 
           * Discount factor: How much to regard future future rewards in comaprison to present rewards
           * Anneal rate: for the deep Q approach, I included a constant rate at which the exploration rate deacresed, as it explored the state space, to limit the effects of random choice as it learned about the state space over time.

## Metrics in training- Game Length
    For measuring the training of data, I the main metric I collected was the average number of rounds played over a hundred game epoch. The baseline is a random agent which choses at random from the possible actions. The baseline agent. The average length of game for a 100-game batch was recorded on ten batches. The results are below.
    
    
### Random. Agent - Game Length

In [ ]:
import pandas as pd
rand_df= pd.read_csv("randomResults.csv")

print("The Average Length of Game for a baseline which makes random moves is: ",rand_df["length"].mean())
print("The Standard deviation from this average across 1000 games was:", rand_df["length"].std())

print(rand_df)

### Q Learning Agent - Game Length 

Q learning made just more than a standard deviation's worth of progress from a random agent.

In [ ]:
q_df = pd.read_csv("qResults.csv")
print("The Average Length of Game for Generalized Q Learning:",q_df["length"].mean())
print("The Standard deviation from this average across 1000 games was:", q_df["length"].std())

print(q_df)

### Deep- Game Length

In [ ]:
deepq_df = pd.read_csv("deepQresults.csv")
print("The Average Length of Game for Generalized Q Learning:",deepq_df["length"].mean())
print("The Standard deviation from this average across 1000 games was:", deepq_df["length"].std())

print(deepq_df)

## All three agent's training performance, graphed

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(deepq_df["round"], deepq_df["length"],
            color='red', marker='o', label='deep Q')

plt.scatter(q_df["iteration"], q_df["length"],
            color='blue', marker='o', label='Q')
plt.xlabel('Batches of 100 games')

plt.ylabel('Average Game Length')

#show benchmark of random action
x_values= [0,300]
y_values =[9.63090909090909, 9.63090909090909]
plt.plot(x_values, y_values)

plt.legend(loc='lower right')
plt.show()

# PART 3: Analysis

There are several key insights as to why the given result occurred. Let's begin with generalized Q learning. The agent did not, in an variation, learn to perform on average more than 10 moves per game. This is only one move better than the randomized result. Why might have this occured? The answer comes down to the features. The features given to the model to analyze the state were: the number of "holes" in the grid, the number of empty white spaces, wether or not the move caused a clearnace of a row, the score for that move, and the total size of the next two peices. From any combiantion of those features, the Q values diverged. Q-values can diverge when the function which it maps to is non-linear according to Maei,Szepesvari,Shalabh Bhatnagar, and Sutton. I beleive this occured because the agent made bold predictions, was wrong by a lot and had a large error, which caused the q-values to oscillate rapidly. Note the values of the wieghts in qlearndata.txt for evidence. So if this is the case, there are several solutions I implmented to prevent this from happening in the Deep-Q agent. 

First is the practice of experince replay. Experince replay can prevent error spikes on local optima, and is widely regarded as good practice. The second is to use a seperate target network. The weights of the neural network responsible for calculating the value of the state reached as a result of an action are frozen in the Target Network, and only periodically copied over from the “online” network. This is another strategy to reduce error, but it can slow down learning. In addition, the neural net used the activation function ReLu or rectfier function. In the context of artificial neural networks, the rectifier is an activation function defined as the positive part of its argument:
f(x)=x^{+}=\max(0,x) where x is the input to a neuron. This is also known as a ramp function. The rectifier is one the most popular activation function for deep neural networks but might help in this case.

The excellent result is that this combination of adaptations allowed the transition from q-learning to deep-q-learning to remove the issue of q-value divergance. In that way, my work was a scucess. In addition, however, we see that the learning going on within the model steadily improved, and then dropped off dramitcally, and then began to improve agian. This odd phenomenon might be related to some kind of overflow-wraparound error, or it might merely just be a small blip in the larger trend that the number of trials (around 1,000,000). One of the key takeways may be that this just needs to train for longer, although it trained for around over 24 hours on a fast enough system. Overall, howeverm the model showed at points marked improvement over the ramdomized model, although perhaps not as astronomically sucessful as I had hoped.

### Future Work:

There are many components which, if implemented, might make for interesting results. Take for instance the component of the Q-model which needed to improve the usefulness of its feature set. Spector, Barto, and Niekum used genetic algorithms to build better reward functions. Perhaps these kinds of genetic algorithms could also sample and test possible features for a generalized q-learning model. For the deep learning mode, the basic outline of that paper could also be implemented in order to improve the reward function, which created based only on the score of the game a penalty for losing the game.

In addition, one further area of improvement is building a more complex neural network. This network, begin rather simple in its construction to abet a better understanding during debugging time, could be made convolutional or recurrent, or just more complex. That way, the network might be able to more effectively model the complex graphical intracices of what makes a good move in Blockuduku. Overall, there are a bunch of specific directions which could improve the performance of the model, and help yield more promisng results. This is not to say that this work is uninteresting, but rather is a good starting point for potenitaly interesting research. The application of chess AIs to produce state-based evaluations I think is a really cool concept to explore in regards to other reenforcment enviroments, as it allows for easy transition between complicated action spaces, and could be very useful for performing complex tasks.

# PART 4 : Try it yourself!

## Deep Q Agent:
This will play the game based on the pickled file "agent.ml" which contains the traained agent, without updating the nueral net. Note that we are using the model as saved at 10,000 games, as its preformance is better.

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation


def visualize(grid):

    fig = plt.figure()
    cmap = mpl.colors.ListedColormap(['white','blue'])
    bounds=[-1,0.5,2]
    norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
    im = plt.imshow(grid, animated=True , interpolation='nearest', cmap = cmap,norm=norm)

    plt.show()

In [ ]:
from deepQEnv import play_agent
move_history = play_agent()

#Note that this is entirely variable based on draws-- sometimes it is not given the opportunity to preform well.
for move in move_history:
    visualize(move)

## Q Agent:

In [ ]:
from qEnv import play_q_agent
play_q_agent()

# Part 5: Sources


Hamid Reza Maei, Csaba Szepesv´ari,Shalabh Bhatnagar, Richard S. Sutton. "Toward Off-Policy Learning Control with Function Approximation"
https://en.wikipedia.org/wiki/Rectifier_(neural_networks)
Matt Stevens, Sabeek Pradhan "Playing Tetris with Deep Reinforcement Learning" http://cs231n.stanford.edu/reports/2016/pdfs/121_Report.pdf
Tom Schaul, John Quan, Ioannis Antonoglou and David Silver "PRIORITIZED EXPERIENCE REPLAY" https://arxiv.org/pdf/1511.05952.pdf
S. Niekum, A. G. Barto and L. Spector, "Genetic Programming for Reward Function Search," in IEEE Transactions on Autonomous Mental Development, vol. 2, no. 2, pp. 83-90, June 2010, doi: 10.1109/TAMD.2010.2051436.